In [7]:
import re
import json
import os
from collections import defaultdict
from itertools import combinations

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
import numpy as np
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec

In [2]:
STOPWORDS = stopwords.words('russian')

In [3]:
# скачаем данные в папке data и распакуем их
# Данные -- ng
PATH_TO_DATA = './data_kw'

In [4]:
files = [os.path.join(PATH_TO_DATA, file) for file in os.listdir(PATH_TO_DATA)]

Объединим файлы в один датасет.

In [5]:
data = pd.concat([pd.read_json(file, lines=True, encoding='utf-8') for file in files], axis=0, ignore_index=True)

In [6]:
data.shape

(1987, 5)

In [15]:
class Corpus:
    
    def __init__(self,
                 vectorizer=None,
                 lemmatize=True,
                 preserve_case=True,
                 parse_named_entities=True,
                 w2v_model=None,
                 allowed_pos_tags=['NOUN']):
        
        self.vectorizer = vectorizer
        self.lemmatize = lemmatize
        self.preserve_case = preserve_case
        self.parse_named_entities = parse_named_entities
        self.w2v_model = w2v_model
        self.allowed_pos_tags = allowed_pos_tags
        
        # named entities
        self.NE = defaultdict(set)
        self.W_2_VECT_IDX= dict()
        self.PREDICT_BY_MAP = {'most_common_ne': self.__predict_most_common_ne,
                               'tfidf': self.__predict_tfidf,
                               'tfidf_ne': self.__predict_tfidf_ne,
                               'centrality': self.__predict_centrality
        }
        self.morph = MorphAnalyzer()
        
    def normalize(self, text, _id=None):
        normalized = list()
        capitalization = list()
        
        for sent in sent_tokenize(text):
            for i, word in enumerate(sent.strip().split()):
                word = re.sub('^[\W]*', '', word)
                word = re.sub('[\W]*$', '', word)

                if not word or word.lower() in STOPWORDS:
                    continue
                
                normalized.append(word.lower())
                capitalization.append(True if word[0].isupper() and i > 0 else False)
        
        if self.lemmatize:
            if self.allowed_pos_tags is not None:
                _normalized = list()
                _capitalization = list()
                
                normalized = [self.morph.parse(word)[0] for word in normalized]
                
                for i, word in enumerate(normalized):
                    if word.tag.POS in self.allowed_pos_tags:
                        _normalized.append(word)
                        _capitalization.append(capitalization[i])
            
                normalized = [word.normal_form for word in _normalized]
                capitalization = _capitalization
            
            else:
                normalized = [self.morph.parse(word)[0].normal_form for word in normalized]
            
        if self.parse_named_entities and _id is not None:
            self.NE[_id] = set([word for i, word in enumerate(normalized) if capitalization[i]])
        
        return ' '.join(normalized)

    def scan_ne(self, text):
        text = text.strip()
        nes = list() 

        for word in text.split()[1:]:
            if word[0].isupper():
                nes.append(word)

        return nes


    def parse_ne(self, docs):
        _map = defaultdict(list)

        for i, doc in enumerate(docs):
            for sent in doc.split():
                _map[i].extend(self.scan_ne(sent))

        return _map
    
    def build(self, documents):
        self.normalized = [self.normalize(doc, i) for i, doc in enumerate(documents)]
        
    def load(self, normalized_path='NORMALIZED', named_entities_path='NE'):
        with open(normalized_path, 'rb') as f:
            self.normalized = pickle.load(f)
        
        if named_entities_path is not None:
            with open(named_entities_path, 'rb') as f:
                self.NE = pickle.load(f)
                
    def fit_vectorizer(self, build_feature_idxs=False):
        self.vectorizer.fit(self.normalized)
        self.features = self.vectorizer.get_feature_names()
        
        #if build_feature_idxs:
        #    for k, v in self.NE.items():
        #        for word in v:
        #            if self.W_2_VECT_IDX.get(word) is None:
        #                try:
        #                    self.W_2_VECT_IDX[word] = self.features.index(word)
        #                
        #                except ValueError:
        #                    continue
        
        if build_feature_idxs:
            for doc in self.normalized:
                for word in doc.split():
                    if self.W_2_VECT_IDX.get(word) is None:
                        try:
                            self.W_2_VECT_IDX[word] = self.features.index(word)
                        
                        except ValueError:
                            continue
                            
        self.features = np.array(self.features)
        
    def __predict_most_common_ne(self, text, _id, n=10):
        nes = [word for word in text.split() if word in self.NE[_id]]
        return [word for word, freq in Counter(nes).most_common(n)]
    
    def __predict_tfidf(self, text, _id, n=10):
        return list(self.features[np.argsort(self.vectorizer.transform([text]).toarray()[0])][-n:])
    
    def __filter_by_cosine(self, predicted, cos_thresh):
        """return words to filter out"""
        
        filtered = set()
        
        for w1, w2 in combinations(predicted, 2):
            pre_filtered = False
            
            try:
                self.w2v_model.wv[w1]
            
            except KeyError:
                filtered.add(w1)
                pre_filtered = True
            
            try:
                self.w2v_model.wv[w2]
            
            except KeyError:
                filtered.add(w2)
                pre_filtered = True

            if not pre_filtered and self.w2v_model.wv.similarity(w1, w2) > cos_thresh:
                filtered.update({w1, w2})

        return predicted.difference(filtered)

    
    def __predict_tfidf_ne(self, text, _id, n=10, coef=1, cosine_threshold=None):
        transformed = self.vectorizer.transform([text]).toarray()[0]
        
        for ne in self.NE[_id]:
            if self.W_2_VECT_IDX.get(ne) is not None:
                transformed[self.W_2_VECT_IDX[ne]] *= coef
        
        predicted = set(self.features[np.argsort(transformed)][-n:])
        
        if cosine_threshold is not None:
            predicted = self.__filter_by_cosine(predicted, cosine_threshold)
            
        return predicted
    
    def __build_graph(self, text, _id, window_size=5, coef=1):
        text = text.split()
        vocab = set(text)
        word2id = {w:i for i, w in enumerate(vocab)}
        id2word = {i:w for i, w in enumerate(vocab)}
        # преобразуем слова в индексы для удобства
        ids = [word2id[word] for word in text]
        
        transformed = self.vectorizer.transform([' '.join(vocab)]).toarray()[0]
        
        id2coef = dict()
        for i, word in id2word.items():
            if self.W_2_VECT_IDX.get(word) is not None:
                if word in self.NE[_id]:
                    id2coef[i] = transformed[self.W_2_VECT_IDX.get(word)] * coef
                
                else:
                    id2coef[i] = transformed[self.W_2_VECT_IDX.get(word)]
            
            else:
                id2coef[i] = 1

        # создадим матрицу совстречаемости
        m = np.zeros((len(vocab), len(vocab)))

        
        #for i in range(0, len(ids), window_size):
        i = 0
        z = window_size
        while i < len(ids):
            window = ids[i:z]
            
            for j, k in combinations(window, 2):
                coef_j = id2coef[j] if id2coef[j] is not None else 1 
                coef_k = id2coef[k] if id2coef[k] is not None else 1
                value = coef_j + coef_k
                
                m[j][k] += value
                m[k][j] += value
            
            i += 1
            z += 1
        
        return m, id2word
    
    def __predict_centrality(self, text, _id, window_size=5, coef=1, n=5):
        graph, id2word = self.__build_graph(text, _id, window_size, coef)
        degrees = dict()
        
        for i in range(graph.shape[0]):
            degrees[i] = graph[i].sum()

        return [id2word[k] for k in sorted(degrees, key=degrees.get)[-n:]]
        
    def predict(self, idxs=None, by='most_common_ne', predictor_kwargs={'n': 10}):
        predictor = self.PREDICT_BY_MAP[by]
        
        if idxs is not None:
            return [predictor(self.normalized[idx], idx, **predictor_kwargs) for idx in idxs]
        
        else:
            return [predictor(doc, i, **predictor_kwargs) for i, doc in enumerate(self.normalized)]
                

def evaluate(true_kws, predicted_kws):
    assert len(true_kws) == len(predicted_kws)
    
    precisions = []
    recalls = []
    f1s = []
    jaccards = []
    
    for i in range(len(true_kws)):
        true_kw = set(true_kws[i])
        predicted_kw = set(predicted_kws[i])
        
        tp = len(true_kw & predicted_kw)
        union = len(true_kw | predicted_kw)
        fp = len(predicted_kw - true_kw)
        fn = len(true_kw - predicted_kw)
        
        if (tp+fp) == 0:
            prec = 0
        else:
            prec = tp / (tp + fp)
        
        if (tp+fn) == 0:
            rec = 0
        else:
            rec = tp / (tp + fn)
        if (prec+rec) == 0:
            f1 = 0
        else:
            f1 = (2*(prec*rec))/(prec+rec)
            
        jac = tp / union
        
        precisions.append(prec)
        recalls.append(rec)
        f1s.append(f1)
        jaccards.append(jac)
    print('Precision - ', round(np.mean(precisions), 2))
    print('Recall - ', round(np.mean(recalls), 2))
    print('F1 - ', round(np.mean(f1s), 2))
    print('Jaccard - ', round(np.mean(jaccards), 2))

Соберем объект, реализующий корпус и весь необходимый функционал

In [8]:
w2v_model = Word2Vec.load(r'E:\w2v\araneum\araneum_none_fasttextskipgram_300_5_2018.model')

In [11]:
corpus = Corpus(vectorizer=TfidfVectorizer(), w2v_model=w2v_model)

In [12]:
%%time
corpus.build(data.content.values)

Wall time: 2min 41s


In [98]:
# load pickled
# corpus.load()

In [13]:
%%time
corpus.fit_vectorizer(True)

Wall time: 9.01 s


В основе моих идей по улучшению качества лежит использование информации об именованных сущностях, потому как в этом датасете таковые составляют большую долю среди всех целевых ключевых слов, а следовательно, при построении модели имеет смысл отдавать им приоритет  
Упрощая, под именованой сущностью я понимал любое не первое в предложении вхождение с заглавной буквы (в целом, несмотря на незамысловатость метода, получилось довольно правдоподобно)  
Перед экспериментами дынные были нормализованы и лемматизированы, а также были удалены стоп-слова

Эксперимент 1  
F1: 0.17 (прирост относительно baseline: +0.01)    

Модификация графового метода: построение графа совместной встречаемости слов в пределе некоторого окна, однако вместо единичек в соответствующую клетку графового представления отправляется сумма tf-idf весов.  
Помимо этого, именованные сущности дополнительно взвешиваются некоторой константной, что тем самым позволяет отдавать им некоторый приоритет (параметр `coef`)    

In [21]:
evaluate(data.keywords.values, corpus.predict(by='centrality', predictor_kwargs={'coef': 2.5}))

Precision -  0.18
Recall -  0.18
F1 -  0.17
Jaccard -  0.1


Эксперимент 2  
F1: 0.18 (прирост относительно baseline: +0.02)
    
Модификации метода, на основе ранжирования по tf-idf весам: в качестве ключевых слов возьмем топ-n слов с наибольшим tf-idf весом, при этом дополнительно взвесим именованные сущности некоторой константой (параметр `coef`) 

In [17]:
evaluate(data.keywords.values, corpus.predict(by='tfidf_ne', predictor_kwargs={'coef': 1.5}))

Precision -  0.14
Recall -  0.28
F1 -  0.18
Jaccard -  0.11


Эксперимент 3  
F1: 0.2 (прирост относительно baseline: +0.04)
    
Вариация эксперимента 3 с отличием в том, что в качестве предсказанных ключевых слов берутся топ-5  

*(ясно, что вариация размера топа не кажется надежным методом, однако в целом оценка качества, предложенная в этом задании, носит в некоторой степени спорный характер, потому как для каждого объекта в датасете число ключевых слов разнится (где-то 3, где-то 20))*

In [18]:
evaluate(data.keywords.values, corpus.predict(by='tfidf_ne', predictor_kwargs={'n': 5, 'coef': 1.5}))

Precision -  0.21
Recall -  0.21
F1 -  0.2
Jaccard -  0.12


Эксперимент 4 (неудачный)

Я также попробовал реализовать предложенную идею фильтрации близких по смыслу слов среди ключевых.  

Для этого я ввел некоторый порог косинусной близости между векторами слов, при превышении которого слова считались близкими по смыслу (были взяты уже построенные векторы с rusvectores).  
Так, в рамках каждого из рассмотренных выше методов к полученному топу применялась описываемая фильтрация, что однако не позволило получить прирост качества (функционал реализован в `Corpus.__filter_by_cosine`)

In [89]:
import pickle

In [90]:
with open('NORMALIZED', 'wb') as f:
    pickle.dump(corpus.normalized, f)
    
with open('NE', 'wb') as f:
    pickle.dump(corpus.NE, f)

## Домашнее задание

В семинаре установлен такой бейзлан - F1 -  0.16 (не будем учитывать точность и полноту по отдельности и отбросим жаккара).

**Ваша задача - предложить 3 способа побить бейзлайн. **

Нет никаких ограничений кроме:

1) нельзя изменять метрику
2) решение должно быть воспроизводимым

В качестве ответа нужно предоставить jupyter тетрадку с экспериментами (обязательное условие!) и описать каждую из идей в форме - https://goo.gl/forms/Zb89yjXFr37EITMq1

Каждый реализованный и описанный способ оценивается в 3 балла. Дополнительный балл можно получить, если способы затрагивают разные аспекты решения (например, первая идея - улучшить нормализацию, вторая - улучшить способ представления текста в виде графа, третья - предложить способ удаления из топа идентичных ключевых слов (рф, россия)).

Можно использовать мой код как основу, а можно придумать что-то полностью другой.

Если у вас никак не получается побить бейзлайн вы можете предоставить реализацию и описание неудавшихся экспериментов (каждый оценивается в 0.5 баллов).